In [1]:
# %%
# https://www.sc-best-practices.org/conditions/gsea_pathway.html#id380
# Kang HM, Subramaniam M, Targ S, et al. Multiplexed droplet single-cell RNA-sequencing using natural genetic variation
#   Nat Biotechnol. 2020 Nov;38(11):1356]. Nat Biotechnol. 2018;36(1):89-94. doi:10.1038/nbt.4042

# %%


import sys
from pathlib import Path

import dotenv
import numpy as np
import pandas as pd
import scanpy as sc
from isrobust.bio import (
    build_hipathia_renamers,
    get_adj_matrices,
    get_reactome_adj,
    sync_gexp_adj,
)
from isrobust.datasets import load_kang
from isrobust.utils import set_all_seeds
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import minmax_scale

model_kind = "binn-reactome"
debug = 1
seed = 42
model_kind = str(model_kind)
debug = bool(int(debug))
seed = int(seed)


project_path = Path(dotenv.find_dotenv()).parent
results_path = project_path.joinpath("results")
results_path.mkdir(exist_ok=True, parents=True)
data_path = project_path.joinpath("data")
data_path.mkdir(exist_ok=True, parents=True)
mygene_path = data_path.joinpath("mygene")
mygene_path.mkdir(exist_ok=True, parents=True)
figs_path = results_path.joinpath("figs")
figs_path.mkdir(exist_ok=True, parents=True)
tables_path = results_path.joinpath("tables")
tables_path.mkdir(exist_ok=True, parents=True)

set_all_seeds(seed=seed)

sc.set_figure_params(dpi=300, color_map="inferno")
sc.settings.verbosity = 1
sc.logging.print_header()

print(f"{debug=} {model_kind=}")


/home/cloucera/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


scanpy==1.9.3 anndata==0.10.3 umap==0.5.5 numpy==1.26.1 scipy==1.11.3 pandas==2.1.1 scikit-learn==1.3.1 statsmodels==0.14.0 pynndescent==0.5.11
debug=True model_kind='binn-reactome'


In [36]:
atlas = sc.read(
        data_path.joinpath("theis", "pbmc_vars_sb.h5ad"),
        cache=False,
    )

atlas = atlas[atlas.obs['study']!='Villani'].copy()
atlas.X = atlas.layers["counts"].copy() 
sc.pp.normalize_total(atlas)
sc.pp.log1p(atlas)


/home/cloucera/github/robustness_informed/.venvs/binn/lib/python3.10/site-packages/anndata/__init__.py:51: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(


In [37]:
atlas.obs

,batch,chemistry,data_type,dpt_pseudotime,final_annotation,mt_frac,n_counts,n_genes,sample_ID,size_factors,species,study,tissue
index,,,,,,,,,,,,,
AAACCTGCAGCGAACA-1-Oetjen_A,Oetjen_A,v2_10X,UMI,NaN,CD16+ Monocytes,0.047970,6379.0,1862.0,0,0.957719,Human,Oetjen,Bone_Marrow
AAACCTGCATGTCCTC-1-Oetjen_A,Oetjen_A,v2_10X,UMI,NaN,CD4+ T cells,0.024928,4172.0,1082.0,0,0.425532,Human,Oetjen,Bone_Marrow
AAACCTGGTCGACTGC-1-Oetjen_A,Oetjen_A,v2_10X,UMI,NaN,CD14+ Monocytes,0.051907,6608.0,1618.0,0,0.773111,Human,Oetjen,Bone_Marrow
AAACCTGGTCGCTTCT-1-Oetjen_A,Oetjen_A,v2_10X,UMI,NaN,CD14+ Monocytes,0.041716,5034.0,1413.0,0,0.641188,Human,Oetjen,Bone_Marrow
AAACCTGTCCCGACTT-1-Oetjen_A,Oetjen_A,v2_10X,UMI,NaN,NKT cells,0.043522,3998.0,1127.0,0,0.452426,Human,Oetjen,Bone_Marrow
...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGTCAAGCTCCTTC-1-Sun_sample4_TC,Sun_sample4_TC,10X,UMI,NaN,CD14+ Monocytes,0.059215,3006.0,1111.0,3,0.825529,Human,Sun,PBMCs
TTTGTCAAGCTGAAAT-1-Sun_sample4_TC,Sun_sample4_TC,10X,UMI,NaN,CD14+ Monocytes,0.051119,5810.0,1723.0,3,1.584353,Human,Sun,PBMCs
TTTGTCATCATCATTC-1-Sun_sample4_TC,Sun_sample4_TC,10X,UMI,NaN,NK cells,0.038078,2705.0,1209.0,3,0.978014,Human,Sun,PBMCs


In [38]:
adata.obs.cell_type

index
AAACATACATTTCC.1    CD14 Mono
AAACATACCAGAAA.1    CD14 Mono
AAACATACCTCGCT.1    CD14 Mono
AAACATACGATGAA.1        CD4 T
AAACATACGGCATT.1    CD14 Mono
                      ...    
TTTGCATGAACGAA.1           DC
TTTGCATGACGTAC.1        CD4 T
TTTGCATGCCTGTC.1            B
TTTGCATGCTAAGC.1        CD4 T
TTTGCATGGGACGA.1        CD4 T
Name: cell_type, Length: 13576, dtype: category
Categories (8, object): ['B', 'CD4 T', 'CD8 T', 'CD14 Mono', 'CD16 Mono', 'DC', 'NK', 'T']

In [39]:

# %%
adata = load_kang(data_folder=data_path, normalize=True, n_genes=4000)

# %%
x_trans = adata.to_df()

obs = adata.obs

/home/cloucera/github/robustness_informed/.venvs/binn/lib/python3.10/site-packages/anndata/__init__.py:51: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(


In [40]:
# %%
circuit_adj, circuit_to_pathway_adj = get_adj_matrices(
    gene_list=x_trans.columns.to_list()
)

circuit_renamer, pathway_renamer, circuit_to_effector = build_hipathia_renamers()

kegg_circuit_names = circuit_adj.rename(columns=circuit_renamer).columns

kegg_pathway_names = circuit_to_pathway_adj.rename(columns=pathway_renamer).columns

circuit_adj.head()

# %%
reactome = get_reactome_adj()
reactome_pathway_names = reactome.columns

x_trans, circuit_adj = sync_gexp_adj(gexp=x_trans, adj=circuit_adj)


In [41]:

# %%
def train_val_test_split(features, val_size, test_size, stratify, seed):
    train_size = 1 - (val_size + test_size)

    x_train, x_test, y_train, y_test = train_test_split(
        features,
        stratify,
        train_size=train_size,
        stratify=stratify,
        random_state=seed,
    )

    x_val, x_test, y_val, y_test = train_test_split(
        x_test,
        y_test,
        test_size=test_size / (test_size + val_size),
        stratify=y_test,
        random_state=seed,
    )

    x_train = x_train.astype("float32")
    x_val = x_val.astype("float32")
    x_test = x_test.astype("float32")

    return x_train, x_val, x_test, y_train, y_val, y_test


In [42]:
from sklearn.preprocessing import LabelEncoder,OneHotEncoder


x_train, x_val, x_test, y_train, y_val, y_test = train_val_test_split(
    x_trans.apply(minmax_scale),
    val_size=0.20,
    test_size=0.20,
    stratify=obs["cell_type"].astype(str) + obs["condition"].astype(str),
    seed=42,
)

y_train = obs["cell_type"][y_train.index]
y_val = obs["cell_type"][y_val.index]
y_test = obs["cell_type"][y_test.index]

label_encoder = OneHotEncoder(sparse_output=False).fit(y_train.to_frame())
y_train_encoded = label_encoder.transform(y_train.to_frame())
y_val_encoded = label_encoder.transform(y_val.to_frame())
y_test_encoded = label_encoder.transform(y_test.to_frame())

In [8]:
from binn import BINN, Network

input_data = pd.read_csv("../../BINN/data/test_qm.csv", sep=",")
translation = pd.read_csv("../../BINN/data/translation.tsv", sep="\t")
pathways = pd.read_csv("../../BINN/data/pathways.tsv", sep="\t").rename(columns={"parent": "source", "child": "target"})

network = Network(
    input_data=input_data,
    pathways=pathways,
    mapping=translation,
)

In [10]:
mygene_path.as_posix

<bound method PurePath.as_posix of PosixPath('/home/cloucera/github/robustness_informed/data/mygene')>

In [11]:
TRANSLATE_GENES = True

if TRANSLATE_GENES:
    import mygene

    mg = mygene.MyGeneInfo()
    mg.set_caching(mygene_path.as_posix())
    gene_trans_df = mg.querymany(translation.input.unique(), scopes="uniprot", fields="symbol", as_dataframe=True, species='human', returnall=True)


gene_trans_df

INFO:biothings.client:[ Future queries will be cached in "/home/cloucera/github/robustness_informed/data/mygene.sqlite" ]
INFO:biothings.client:querying 1-1000...
INFO:biothings.client:done.
INFO:biothings.client:querying 1001-2000...
INFO:biothings.client:done.
INFO:biothings.client:querying 2001-3000...
INFO:biothings.client:done.
INFO:biothings.client:querying 3001-4000...
INFO:biothings.client:done.
INFO:biothings.client:querying 4001-5000...
INFO:biothings.client:done.
INFO:biothings.client:querying 5001-6000...
INFO:biothings.client:done.
INFO:biothings.client:querying 6001-7000...
INFO:biothings.client:done.
INFO:biothings.client:querying 7001-8000...
INFO:biothings.client:done.
INFO:biothings.client:querying 8001-9000...
INFO:biothings.client:done.
INFO:biothings.client:querying 9001-10000...
INFO:biothings.client:done.
INFO:biothings.client:querying 10001-11000...
INFO:biothings.client:done.
INFO:biothings.client:querying 11001-11613...
INFO:biothings.client:done.
INFO:biothin

{'out':            notfound     _id     _score   symbol
 query                                          
 A0A075B6P5     True     NaN        NaN      NaN
 A0A075B6S6     True     NaN        NaN      NaN
 A0A096LP49      NaN  399693  19.768894  CCDC187
 A0A0A6YYK7     True     NaN        NaN      NaN
 A0A0C4DH25     True     NaN        NaN      NaN
 ...             ...     ...        ...      ...
 Q9Y6X5          NaN   22875  19.768894    ENPP4
 Q9Y6X9          NaN   22880  19.768894    MORC2
 Q9Y6Y8          NaN   11196  19.768894  SEC23IP
 Q9Y6Y9          NaN   23643  19.768894     LY96
 Q9Y6Z7          NaN   10584  19.768894  COLEC10
 
 [11737 rows x 4 columns],
 'dup':      query  duplicate hits
 0   O14950               2
 1   O15263               2
 2   O43508               2
 3   O75022               2
 4   O95925               2
 ..     ...             ...
 82  Q9ULR0               2
 83  Q9Y3B3               2
 84  Q9Y3E7               2
 85  Q9Y575               2
 86  Q9Y6Q2 

In [13]:
translation = translation.merge(gene_trans_df["out"].dropna(subset="symbol").reset_index(names="input")[["input", "symbol"]],
                  how="left",
                  ).dropna(subset="symbol").drop(["input", "Unnamed: 0"], axis=1).rename(columns={"symbol": "input"})

translation

,translation,input
52,R-HSA-9013106,CCDC187
111,R-HSA-6809371,LCE6A
112,R-HSA-5620924,IFT56
113,R-HSA-5620924,IFT56
114,R-HSA-8866654,TMEM129
...,...,...
51360,R-HSA-937072,LY96
51361,R-HSA-9707616,LY96
51362,R-HSA-975163,LY96
51363,R-HSA-166662,COLEC10


In [23]:
genes = x_trans.columns.intersection(translation.input)

translation = translation.loc[translation.input.isin(genes)]
x_trans = x_trans.loc[:, genes]

In [14]:
kegg_trans = circuit_adj.melt(ignore_index=False).reset_index(names="symbol").query("value>0").rename(columns={"symbol": "source", "circuit": "target"}).drop("value", axis=1)

kegg_trans.head()

,source,target
1063,PPARD,P-hsa03320-62
1267,RXRA,P-hsa03320-62
1268,RXRB,P-hsa03320-62
1454,UBC,P-hsa03320-62
2801,PDPK1,P-hsa03320-45


In [ ]:
network = Network(
    input_data=x_trans.T.reset_index(names="gene"),
    pathways=pathways,
    input_data_column="gene"
)

In [24]:
from sklearn.preprocessing import LabelEncoder,OneHotEncoder


x_train, x_val, x_test, y_train, y_val, y_test = train_val_test_split(
    x_trans.apply(minmax_scale),
    val_size=0.20,
    test_size=0.20,
    stratify=obs["cell_type"].astype(str) + obs["condition"].astype(str),
    seed=42,
)

y_train = obs["cell_type"][y_train.index]
y_val = obs["cell_type"][y_val.index]
y_test = obs["cell_type"][y_test.index]

label_encoder = OneHotEncoder(sparse_output=False).fit(y_train.to_frame())
y_train_encoded = label_encoder.transform(y_train.to_frame())
y_val_encoded = label_encoder.transform(y_val.to_frame())
y_test_encoded = label_encoder.transform(y_test.to_frame())

In [ ]:

batch_size = 32

callback = callbacks.EarlyStopping(
    monitor="val_loss",  # Stop training when `val_loss` is no longer improving
    min_delta=1e-1,  # "no longer improving" being defined as "no better than 1e-5 less"
    patience=100,  # "no longer improving" being further defined as "for at least 3 epochs"
    verbose=0,
)

vae, encoder, decoder = build_kegg_vae(
        circuits=circuit_adj, pathways=circuit_to_pathway_adj, seed=seed
    )

history = vae.fit(
    x_train.values,
    shuffle=True,
    verbose=0,
    epochs=N_EPOCHS,
    batch_size=batch_size,
    callbacks=[callback],
    validation_data=(x_val.values, None),
)

In [25]:
network = Network(
    input_data=x_trans.T.reset_index(names="gene"),
    pathways=pathways,
    mapping=translation,
    input_data_column="gene"
)

In [26]:
from binn import BINNClassifier, BINN
import torch 
from torch.utils.data import DataLoader, TensorDataset
import lightning.pytorch as pl

device = torch.device("cuda")

binn = BINN(
    n_layers=2,
    network=network,
    n_outputs=y_train_encoded.shape[1],
    device="cuda"
)

binn


BINN is on the device: cuda:0


BINN(
  (layers): Sequential(
    (Layer_0): Linear(in_features=917, out_features=172, bias=True)
    (BatchNorm_0): BatchNorm1d(172, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (Dropout_0): Dropout(p=0, inplace=False)
    (Tanh 0): Tanh()
    (Layer_1): Linear(in_features=172, out_features=542, bias=True)
    (BatchNorm_1): BatchNorm1d(542, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (Dropout_1): Dropout(p=0, inplace=False)
    (Tanh 1): Tanh()
    (Output layer): Linear(in_features=542, out_features=8, bias=True)
  )
  (loss): CrossEntropyLoss()
)

In [27]:
import os
n_workers = os.cpu_count() - 1
n_workers

63

In [28]:
dataloader = DataLoader(
            dataset=TensorDataset(torch.Tensor(x_train.values), torch.Tensor(y_train_encoded)),
            batch_size=32,
            shuffle=True,
            num_workers=n_workers
        )


In [29]:
x_train.values.shape

(14803, 1650)

In [30]:

trainer = pl.Trainer(
    callbacks=[],
    max_epochs=10,
)

trainer.fit(binn, dataloader)

Trainer will use only 1 of 3 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=3)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]

  | Name   | Type             | Params
--------------------------------------------
0 | layers | Sequential       | 257 K 
1 | loss   | CrossEntropyLoss | 0     
--------------------------------------------
257 K     Trainable params
0         Non-trainable params
257 K     Total params
1.030     Total estimated model params size (MB)


Epoch 0:   0%|          | 0/463 [00:00<?, ?it/s] 

RuntimeError: mat1 and mat2 shapes cannot be multiplied (32x1650 and 917x172)

In [ ]:



def get_importances(data, abs=False):
    if abs:
        return np.abs(data).mean(axis=0)
 v   else:
        return data.mean(axis=0)


def get_actiations(act_model, layer_id, data):
    data_encoded = act_model.predict(data)[layer_id]
    return data_encoded


# %%
def train_val_test_split(features, val_size, test_size, stratify, seed):
    train_size = 1 - (val_size + test_size)

    x_train, x_test, y_train, y_test = train_test_split(
        features,
        stratify,
        train_size=train_size,
        stratify=stratify,
        random_state=seed,
    )

    x_val, x_test = train_test_split(
        x_test,
        test_size=test_size / (test_size + val_size),
        stratify=y_test,
        random_state=seed,
    )

    x_train = x_train.astype("float32")
    x_val = x_val.astype("float32")
    x_test = x_test.astype("float32")

    return x_train, x_val, x_test


In [ ]:


# %%
results_path_model = results_path.joinpath(model_kind)
obs = adata.obs.copy()
results_path_model.mkdir(exist_ok=True, parents=True)


In [ ]:

# %%
x_train, x_val, x_test = train_val_test_split(
    x_trans.apply(minmax_scale),
    val_size=0.20,
    test_size=0.20,
    stratify=obs["cell_type"].astype(str) + obs["condition"].astype(str),
    seed=42,
)

if model_kind == "ivae_kegg":
    vae, encoder, decoder = build_kegg_vae(
        circuits=circuit_adj, pathways=circuit_to_pathway_adj, seed=seed
    )
elif model_kind == "ivae_reactome":
    vae, encoder, decoder = build_reactome_vae(reactome, seed=seed)
else:
    raise NotImplementedError("Model not yet implemented.")

batch_size = 32

callback = callbacks.EarlyStopping(
    monitor="val_loss",  # Stop training when `val_loss` is no longer improving
    min_delta=1e-1,  # "no longer improving" being defined as "no better than 1e-5 less"
    patience=100,  # "no longer improving" being further defined as "for at least 3 epochs"
    verbose=0,
)

history = vae.fit(
    x_train.values,
    shuffle=True,
    verbose=0,
    epochs=N_EPOCHS,
    batch_size=batch_size,
    callbacks=[callback],
    validation_data=(x_val.values, None),
)

evaluation = {}
evaluation["train"] = vae.evaluate(
    x_train, vae.predict(x_train), verbose=0, return_dict=True
)
evaluation["val"] = vae.evaluate(x_val, vae.predict(x_val), verbose=0, return_dict=True)
evaluation["test"] = vae.evaluate(
    x_test, vae.predict(x_test), verbose=0, return_dict=True
)

pd.DataFrame.from_dict(evaluation).reset_index(names="metric").assign(seed=seed).melt(
    id_vars=["seed", "metric"],
    value_vars=["train", "val", "test"],
    var_name="split",
    value_name="score",
).assign(model=model_kind).to_pickle(
    results_path_model.joinpath(f"metrics-seed-{seed:02d}.pkl")
)

layer_outputs = [layer.output for layer in encoder.layers]
activation_model = Model(inputs=encoder.input, outputs=layer_outputs)

# only analyze informed and funnel layers
for layer_id in range(1, len(layer_outputs)):
    if model_kind == "ivae_kegg":
        if layer_id == 1:
            colnames = kegg_circuit_names
            layer_name = "circuits"
        elif layer_id == 2:
            colnames = kegg_pathway_names
            layer_name = "pathways"
        elif layer_id == (len(layer_outputs) - 1):
            n_latents = len(kegg_pathway_names) // 2
            colnames = [f"latent_{i:02d}" for i in range(n_latents)]
            layer_name = "funnel"
        else:
            continue
    elif model_kind == "ivae_reactome":
        if layer_id == 1:
            colnames = reactome_pathway_names
            layer_name = "pathways"
        elif layer_id == (len(layer_outputs) - 1):
            n_latents = len(reactome_pathway_names) // 2
            colnames = [f"latent_{i:02d}" for i in range(n_latents)]
            layer_name == "funnel"
        else:
            continue
    else:
        raise NotImplementedError("Model not yet implemented.")

    print(f"encoding layer {layer_id}")

    encodings = get_activations(
        act_model=activation_model,
        layer_id=layer_id,
        data=x_trans.apply(minmax_scale),
    )
    encodings = pd.DataFrame(encodings, index=x_trans.index, columns=colnames)
    encodings["split"] = "train"
    encodings.loc[x_val.index, "split"] = "val"
    encodings.loc[x_test.index, "split"] = "test"
    encodings["layer"] = layer_name
    encodings["seed"] = seed
    encodings["model"] = model_kind
    encodings = encodings.merge(
        obs[["cell_type", "condition"]],
        how="left",
        left_index=True,
        right_index=True,
    )
    encodings.to_pickle(
        results_path_model.joinpath(
            f"encodings_layer-{layer_id:02d}_seed-{seed:02d}.pkl"
        )
    )
